## Making tools

### Using technique

In [1]:
## Regular Expression

# s.find('substring') return the indice the substring if exists else -1
# @pattern, origin        ----> pattern 
#           baa!|baaa!    ----> baa*!
# '*' : any numbers of the preceding word
# '?' : zero or one word(exist or not exist)
# '.' : any single character
# '^' : the begging of the text
# '$' : the end of the text

def search(pattern, text):
    "Return True if pattern appears anywhere in text"
    if pattern.startswith('^'):
        return match(pattern[1:] , text)
    else:
#         print('.*' + pattern)
        return match('.*' + pattern, text)
    
def match(pattern, text):
    "return True if pattern appears at the start of text"
    if pattern == '':
        return True
    
    elif pattern == '$':
        return (text == '')
    elif len(pattern) > 1 and pattern[1] in '*?':
        p, op, pat = pattern[0], pattern[1], pattern[2:]
#         print(p, op, pat)
        if op == '*':
            return match_star(p, pat, text)
        elif op == '?':
            if match1(p, text) and match(pat, text):
                return True
            else:
                return match(pat, text)
        
    else:
        return (match1(pattern[0], text) and match(pattern[1:], text[1:]))
    

def match1(p, text):
    """Return True if first character of text natches pattern character p"""
    if not text: return False
    return p == '.' or p == text[0]


def match_star(p, pattern, text):
    """Return true if any number of char p, follwed by pattern, matches text."""
    return (match(pattern, text) or
            (match1(p, text) and match_star(p, pattern, text[1:])))
    
def test():
    #test search(pattern, text)
    assert search('baa*!', 'Sheep said baaaa!') == True
    assert search('baa*!', 'Sheep said baaaa humbu!') == False
    assert search('def', 'abcdefg')
    assert search('def$', 'abcdef')
    assert search('def$', 'abcdefg') == False
    assert search('^start', 'not the start') == False
    assert search('start', 'not the start')
    
    # test match(pattern ,text)
    assert match('baa*!', 'Sheep said baaa!') == False
    assert match('baa*!', 'baaaaaaa! said the sheep')
    assert match('start', 'not the start') == False
    assert match('a*b*c*', 'just anything')
    assert match('x?', 'text')
    assert match('text?', 'text')
    assert match('text?', 'tex')
    
    assert all(match('aa*bb*cc*$', s) 
               for s in words('abc aaabbcc aaaabcccc'))
    assert not any(match('aa*bb*cc*$', s)
                   for s in words('ac aaabbcccd aaaa-b-cccc'))
    assert all(search('^ab.*aca.*a$', s)
              for s in words('abracadavra abacaa about-acacia-a'))
    assert all(search('t.p', s)
              for s in words('tip top tap atypical tepid stop'))
    assert not any(search('t.p', s)
                  for s in words('TYPE teepee tp'))
    
    return 'test passes'
    
def words(text):return text.split()


print(test())

## http://tinyurl.com/pike-regexp

test passes


### Language && Grammer
**language** : a set of strings  
**grammer** : a description of a language

#### Regular Expression

In [2]:
import re

In [3]:
def search(pattern, text):
    "Match pattern anywhere in text; return longest earliest match or None"
    for i in range(len(text)):
        m = match(pattern, text[i:])
        if m is not None:
            return m
def match(pattern, text):
    "Match pattern againest start of text; return longest match found or None"
    remainders = matchset(pattern ,text)
    if remainders:
        shortest = min(remainders, key = len)
        return text[:len(text) - len(shortest)]

def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y

def matchset(pattern, text):
    "Match pattern at start of text; return a set of ramainders of text."
    op, x ,y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]] if text else null)
    elif 'oneof' == op:
        return set([text[1:]]) if any(text.startswith(c) for c in x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                   for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: {}'.format(pattern))
        
null = frozenset()

def lit(string):
    return ('lit', string)

def seq(x,y):
    return ('seq', x, y)

def alt(x, y):
    return ('alt', x, y)

def star(x):
    return ('star', x)
   
def plus(x):
    return ('plus', x)

def opt(x):
    return ('opt', x)

def oneof(chars):
    return ('oneof', tuple(chars))

dot = ('dot',)

eol = ('eol',)

def test_search():
    a,b,c = lit('a'), lit('b'), lit('c')
    abcstars = seq(star(a), seq(star(b), star(c)))
    dotstar = star(dot)
    r = seq(lit('ab'), seq(dotstar, seq(lit('aca'), seq(dotstar, seq(a, eol))))) 
    
    assert search(lit('def'), 'abcdefg') == 'def'
    assert search(seq(lit('def'), eol), 'abcdef') == 'def'
    assert search(seq(lit('def'), eol), 'abcdefg') == None
    assert search(a, 'not the start') == 'a'
    
    assert match(a, 'not the start') == None
    assert match(abcstars, 'aaabbbccccccccdef') == 'aaabbbcccccccc'
    assert match(abcstars, 'junk') == ''
    
    assert all(match(seq(abcstars, eol), s) == s
              for s in 'abc aaabbccc aaaabcccc'.split())
    assert all(match(seq(abcstars, eol), s) == None
              for s in 'cab aaabbcccd aaaa-b-cccc'.split())
    assert all(search(r, s) is not None
              for s in 'abracadabra abacaa about-acacia-flora'.split())
    assert all(match(seq(c, seq(dotstar, b)), s) is not None
              for s in 'cab cab carob cb carbuncle'.split())
    assert not any(match(seq(c, seq(dot,b)), s) 
                  for s in 'crab cb across scab'.split())
    
    return 'test_search passes'

In [4]:
print (test_search())

test_search passes


In [5]:
matchset(seq(dot, lit('2')), '123')

{'3'}

### compiler && interpreter

In [6]:
def match(pattern ,text):
    "Match pattern against start of text"
    remainders = pattern(text)
    if remainders:
        shortest = min(remainders, key = len)
        return text[:len(text) - len(shortest)]


def lit(string):
    return lambda text:set([text[len(string):]]) if text.startswith(string) else null

def seq(x,y):
    return lambda text:set().union(*map(y, x(text)))

def alt(x, y):
    return lambda text: x(text) | y(text)

def star(x):
    return lambda text: (set([text]) | 
                        set(t2 for t1 in x(text) if t1 != text
                           for t2 in star(x)(t1)))
def plus(x):
    return ('plus', x)

def opt(x):
    return ('opt', x)

def oneof(chars):
    return lambda text: set(text[1:]) if (t and t[0] in chars) else null

dot = lambda text: set([text[1:]]) if text else null

eol = lambda text: set[''] if t == '' else null

In [9]:
pat = star(lit('a'))
pat('bc')

{'bc'}

In [10]:
import dis
dis.dis(lambda x, y: aqrt(x ** 2 + y ** 2))

  2           0 LOAD_GLOBAL              0 (aqrt)
              2 LOAD_FAST                0 (x)
              4 LOAD_CONST               1 (2)
              6 BINARY_POWER
              8 LOAD_FAST                1 (y)
             10 LOAD_CONST               1 (2)
             12 BINARY_POWER
             14 BINARY_ADD
             16 CALL_FUNCTION            1
             18 RETURN_VALUE


### Generatos 

In [11]:
## Ns: number set

def lit(s):
    set_s = set([s])          # avoid repetition
    return lambda Ns:set_s if len(s) in Ns else null

def seq(x,y):
    return lambda Ns: genseq(x, y, Ns)

def alt(x, y):
    return lambda Ns: x(Ns) | y(Ns)

def star(x):
    return lambda Ns: opt(plus(x))(Ns)

def plus(x):
    return lambda Ns: genseq(x, star(x), Ns, startx = 1)

def opt(x):
    return alt(epsilon, x)

def oneof(chars):
    set_chars = set(chars)
    return lambda Ns: set_chars if 1 in Ns else null

dot = oneof('?')              ## You should expand the alphabet to more chars
epsilon = lit('')            ## The pattern that matches the empty string


def genseq(x, y, Ns):
    Nss = range(max(Ns) + 1)
    return set(m1 + m2
              for m1 in x(Nss)
              for m2 in y(Nss)
              if len(m1 + m2) in Ns)

def genseq(x, y, Ns, startx = 0):
    "Set of matches to xy whose total length is Ns, witn x-match's length is Ns_x"
    # Tricky part: x+ is defined as: x+ = x x*
    # To stop the recursion, the first x must generate at least 1 char
    # and then the recursive x* has that many fewer characters. We use
    # startx = 1 to say that x must at least 1 character
    if not Ns:
        return null
    
    xmatches = x(set(range(startx, max(Ns) + 1)))
#     print(xmatches)
    Ns_x = set(len(m) for m in xmatches)
    Ns_y = set(n - m for n in Ns for m in Ns_x if n - m >=0)
    ymatches = y(Ns_y)
#     print(ymatches)
    return set(m1 + m2
              for m1 in xmatches
              for m2 in ymatches
              if len(m1 + m2) in Ns)

def N(hi): return set(range(hi + 1))

def test_gen():
    a,b,c = map(lit, 'abc')
    assert star(oneof('ab'))(N(2)) == set(['', 'a', 'aa', 'ab','b','ba', 'bb'])
    assert (seq(star(a), seq(star(b), star(c)))(set([4])) == 
    set(['aaaa', 'aaab', 'aaac', 'aabb', 'aabc', 'aacc', 'abbb', 'abbc', 'abcc', 'accc', 'bbbb',
         'bbbc', 'bbcc', 'bccc', 'cccc']))
    assert (seq(plus(a), seq(plus(b), plus(c)))(set([5])) == 
            set(['aaabc', 'aabbc', 'aabcc', 'abbbc', 'abbcc', 'abccc']))
    assert (seq(oneof('bcfhrsm'), lit('at'))(N(3)) ==
           set(['bat', 'cat', 'fat', 'hat', 'mat', 'rat', 'sat']))
    assert (seq(star(alt(a, b)), opt(c))(set([3])) == 
           set(['aaa', 'aab', 'aac', 'aba', 'abb', 'abc', 'baa', 'bab', 'bac', 'bba', 'bbb', 'bbc']))
    assert lit('hello')(set([5])) == set(['hello'])
    assert lit('hello')(set([4])) == set()
    assert lit('hello')(set([6])) == set()
    return "test_gen passes"

In [12]:
print(test_gen())

test_gen passes


### Change Seq
- backward compatible
- the change is internal or external
    - the inside of sequence doesn't efect all the callers
    - 

### Decorator 

In [13]:
from functools import update_wrapper

def decorator(d):
    "Make function d a decorator: d wraps a function to fn"
    def _d(fn):
        return update_wrapper(d(fn), fn)
    
    update_wrapper(_d, d)
    return _d

@decorator
def n_ary(f):
    """Given binary function(x, y), return an n_ary function such that 
    f(x,y,z) = f(x, f(y,z)),etc.Also allow f(x) = x."""
    def n_ary_f(x, *args):
        return x if not args else f(x, n_ary(*args))
    
#     update_wrapper(n_ary_f, f)
    return n_ary_f


## DECORATOR


@n_ary
def seq(x, y): return ('seq', x, y)

## equal @n_ary
seq = n_ary(seq)

help(seq)

Help on function seq in module __main__:

seq(x, y)



In [14]:
@decorator
def memo(f):
    """Decorator that caches the return value for each call to f(args).
    Then when called again with same args, we can just look it up"""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args cann't be a dict key
            return f(args)
    return _f

@decorator
def countcall(f):
    "Decorator that make the function count calls to it, in callcounts[f]"
    def _f(*args):
        callcounts[_f] += 1
        return f(*args)
    callcounts[_f] = 0
    return _f

callcounts = {} 

@decorator
def trace(f):
    indent = '    '
    def _f(*args):
        signature = "{}({})".format(f.__name__, ','.join(map(repr, args)))
        print("{}--> {}".format(trace.level*indent, signature))
        trace.level += 1
        try:
#             print("before{}".format(args))
            result = f(*args)
#             print(args)
            print("{}<-- {} === {}".format((trace.level - 1)*indent, signature, result))
            
        finally:
                trace.level -= 1
        
        return result
    trace.level = 0
    return _f

@decorator
def disabled(f):
    return f

@trace
@countcall
@memo 
def fib(n): return 1 if n <= 1 else fib(n-1) + fib(n-2)

- Dubugging Tools: countcalls, trace
- Performance Tools: memo
- Expressiveness Tools: n_ary and another

In [16]:
fib(6)

--> fib(6)
    --> fib(5)
        --> fib(4)
            --> fib(3)
                --> fib(2)
                    --> fib(1)
                    <-- fib(1) === 1
                    --> fib(0)
                    <-- fib(0) === 1
                <-- fib(2) === 2
                --> fib(1)
                <-- fib(1) === 1
            <-- fib(3) === 3
            --> fib(2)
            <-- fib(2) === 2
        <-- fib(4) === 5
        --> fib(3)
        <-- fib(3) === 3
    <-- fib(5) === 8
    --> fib(4)
    <-- fib(4) === 5
<-- fib(6) === 13


13

### Grammer 

In [15]:
def split(text, sep = None, maxsplit = -1):
    "Like str.split applied to text, but strips whitespace from each piece"
    return [t.strip() for t in text.strip().split(sep, maxsplit) if t]

def grammar(description, whitespace=r'\s*'):
    """Convert a discription to a grammar"""
    G = {' ': whitespace}
    description = description.replace('\t', ' ') # no tabs!
    for line in split(description, '\n'):
        lhs, rhs = split(line, ' => ', 1)
        alternatives = split(rhs, ' | ')
        G[lhs] = tuple(map(split, alternatives))
    return G

G = grammar(r"""
Exp         => Term [+-] Exp | Term
Term        => Factor [*/] Term | Factor
Factor      => Funcall | Var | Num | [(] Exp [)]
Funcall     => Var [(] Exps [)]
Exps        => Exp [,] Exps | Exp
Var         => [a-zA-Z]\w*
Num         => [-+]?[0-9]+([.][0-9]*)?
""")

In [ ]:
G

In [17]:
def parse(start_symbol, text, grammer):
    """Example call: parse('Exp', '3*x + b', G)
    Returns a (tree,remainder)pair. If remainder is '', it parsed the whole
    string. Failure if ramainder is None, This is a deterministic PEG parser,
    so rule order (left-to-right) matters. Do 'E => T op E | T', putting the 
    longest parse first; don't do 'E => T | T op E'
    Alse, no left recursion allowed: don't do 'E => E op T'"""
    tokenizer = grammer[' '] + '(%s)'
    
    def parse_sequence(sequence, text):
        result = []
        for atom in sequence:
            tree, text = parse_atom(atom, text)
            if text is None: return Fail
            result.append(tree)
        return result, text
    
    @memo
    def parse_atom(atom, text):
        if atom in grammer:
            for alternative in grammer[atom]:  # Non-Terminal: tuple or alternatives
                tree, rem = parse_sequence(alternative, text)
                if rem is not None: return [atom] + tree, rem
            return Fail
        else: # Terminal: match characters against start of text
            m = re.match(tokenizer % atom, text)
            return Fail if (not m) else (m.group(1), text[m.end():])
        
    # Body of parse
    return parse_atom(start_symbol, text)

Fail = (None, None)

In [18]:
def varify(G):
    lhstokens = set(G) - set([' '])
    rhstokens = set(t for alts in G.values() for alt in alts for t in alt)
    def show(title, tokens):
        print(title + '=' + ' '.join(sorted(tokens)))
        
    show('Non-Terms', G)
    show('Terminals', rhstokens - lhstokens)
    show('Suspects', [t for t in (rhstokens - lhstokens) if t.isalnum()])
    show('Orphans ', lhstokens - rhstokens)

### Problem Set 03 

#### Json Parser 

In [21]:
# ---------------
# User Instructions
#
# In this problem, you will be using many of the tools and techniques
# that you developed in unit 3 to write a grammar that will allow
# us to write a parser for the JSON language. 
#
# You will have to visit json.org to see the JSON grammar. It is not 
# presented in the correct format for our grammar function, so you 
# will need to translate it.

# ---------------

JSON = grammar("""
object => { } | { members }
members => pair , members | pair
pair => string : value
array => [[] []] | [[] elements []]
elements => value , elements | value
value => string | number | object | array | true | false | null
string => "[^"]*"
number => int frac exp | int frac | int exp | int
int => -?[1-9][0-9]*
frac => [.][0-9]+
exp => [eE][-+]?[0-9]+
""", whitespace='\s*')

def json_parse(text):
    return parse('value', text, JSON)

def test():
    assert json_parse('["testing", 1, 2, 3]') == (                      
                       ['value', ['array', '[', ['elements', ['value', 
                       ['string', '"testing"']], ',', ['elements', ['value', ['number', 
                       ['int', '1']]], ',', ['elements', ['value', ['number', 
                       ['int', '2']]], ',', ['elements', ['value', ['number', 
                       ['int', '3']]]]]]], ']']], '')
    
    assert json_parse('-123.456e+789') == (
                       ['value', ['number', ['int', '-123'], ['frac', '.456'], ['exp', 'e+789']]], '')
    
    assert json_parse('{"age": 21, "state":"CO","occupation":"rides the rodeo"}') == (
                      ['value', ['object', '{', ['members', ['pair', ['string', '"age"'], 
                       ':', ['value', ['number', ['int', '21']]]], ',', ['members', 
                      ['pair', ['string', '"state"'], ':', ['value', ['string', '"CO"']]], 
                      ',', ['members', ['pair', ['string', '"occupation"'], ':', 
                      ['value', ['string', '"rides the rodeo"']]]]]], '}']], '')
    return 'tests pass'

In [23]:
print(test())

tests pass


#### Inverse Function 

In [29]:
# --------------
# User Instructions
#
# Write a function, inverse, which takes as input a monotonically
# increasing (always increasing) function that is defined on the 
# non-negative numbers. The runtime of your program should be 
# proportional to the LOGARITHM of the input. You may want to 
# do some research into binary search and Newton's method to 
# help you out.
#
# This function should return another function which computes the
# inverse of the input function. 
#
# Your inverse function should also take an optional parameter, 
# delta, as input so that the computed value of the inverse will
# be within delta of the true value.

# -------------
# Grading Notes
#
# Your function will be called with three test cases. The 
# input numbers will be large enough that your submission
# will only terminate in the allotted time if it is 
# efficient enough. 

def slow_inverse(f, delta=1/128.):
    """Given a function y = f(x) that is a monotonically increasing function on
    non-negatve numbers, return the function x = f_1(y) that is an approximate
    inverse, picking the closest value to the inverse, within delta."""
    def f_1(y):
        x = 0
        while f(x) < y:
            x += delta
        # Now x is too big, x-delta is too small; pick the closest to y
        return x if (f(x)-y < y-f(x-delta)) else x-delta
    return f_1 

def inverse(f, delta = 1/128.):
    """Given a function y = f(x) that is a monotonically increasing function on
    non-negatve numbers, return the function x = f_1(y) that is an approximate
    inverse, picking the closest value to the inverse, within delta."""
    def f_1(y):
        lo, hi = find_bound(f, y)
        return binary_search(f, y, lo, hi, delta)
    return f_1

def find_bound(f, y):
    "Find values lo, hi such that f(lo) <= y <= f(hi)"
    # Keep doubling x until f(x) >= y; that's hi;
    # and lo will be either the previous x or 0.
    x = 1
    while f(x) < y:
        x = x * 2.
    lo = 0 if (x == 1) else x/2
    return lo, x

def binary_search(f, y, lo, hi, delta):
    "Given f(lo) <= y <= f(hi), return x such that f(x) is within delta of y."
    while lo <= hi:
        x = (lo + hi)/2
        if f(x) < y:
            lo = x + delta
        elif f(x) > y:
            hi = x - delta
        else:
            return x;
    return hi if (f(hi) - y < y - f(lo)) else lo

def square(x): return x*x

def pow10(x): return 10**x

log10 = inverse(pow10)

sqrt = slow_inverse(square)

print (sqrt(1000000000))

print(log10(100000000))

31622.7734375
8.007781982421875
